In [15]:
import pandas as pd
import numpy as np
from datetime import datetime

import plotly.graph_objects as go

In [110]:
# Import xlsx file and store each sheet in to a df list
xl_file = pd.ExcelFile('./data.xlsx',)

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [111]:
# Data from each sheet can be accessed via key
dfs.keys()

dict_keys(['2020-01-29-14-30', '2020-01-29-13-00', '2020-01-28-23-00', '2020-01-28-18-00', '2020-01-28-13-00', '2020-01-27-20-30', '2020-01-27-19-00', '2020-01-27-09-00', '2020-01-26-23-00', '2020-01-26-11-00', '2020-01-25-22-00', '2020-01-25-12-00', '2020-01-25-00-00', '2020-01-24-12-00', '2020-01-24-00-00', '2020-01-23-12-00', '2020-01-22-12-00', '2020-01-22-00-00'])

In [112]:
dfs['2020-01-28-23-00'].head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,Hubei,Mainland China,1/28/2020 23:00,3554,125.0,80.0
1,Zhejiang,Mainland China,1/28/2020 23:00,296,NaN,3.0
2,Guangdong,Mainland China,1/28/2020 23:00,241,NaN,5.0
3,Hunan,Mainland China,1/28/2020 23:00,221,NaN,NaN
4,Henan,Mainland China,1/28/2020 23:00,206,2.0,1.0


In [113]:
# Data cleansing
for key, df in dfs.items():
    dfs[key].loc[:,'Confirmed'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Deaths'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Recovered'].fillna(value=0, inplace=True)
    dfs[key]=dfs[key].astype({'Confirmed':'int64', 'Deaths':'int64', 'Recovered':'int64'})
    # Add a zero to the date so can be convert by datetime.strptime as 0-padded date
    dfs[key]['Last Update'] = '0' + dfs[key]['Last Update']
    dfs[key]['Date_last_updated'] = [datetime.strptime(d, '%m/%d/%Y %H:%M') for d in dfs[key]['Last Update']]

In [115]:
# Check 
dfs['2020-01-24-00-00'].head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Suspected,Date_last_updated
0,Hubei,Mainland China,01/24/2020 00:00,549,24,31,NaN,2020-01-24
1,Guangdong,Mainland China,01/24/2020 00:00,53,0,2,NaN,2020-01-24
2,Zhejiang,Mainland China,01/24/2020 00:00,43,0,1,NaN,2020-01-24
3,Beijing,Mainland China,01/24/2020 00:00,26,0,0,22.0,2020-01-24
4,Shanghai,Mainland China,01/24/2020 00:00,20,0,0,NaN,2020-01-24


In [116]:
if dfs['2020-01-24-00-00']['Date_last_updated'][1] < dfs['2020-01-24-12-00']['Date_last_updated'][1]:
    print('Yes')

Yes


In [117]:
dfs['2020-01-24-00-00']['Date_last_updated'][1].date()

datetime.date(2020, 1, 24)

In [105]:
dfTpm = dfs['2020-01-29-14-30'].groupby(['Country/Region'])['Confirmed'].agg(np.sum)
dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Confirmed':dfTpm.values})
dfTpm = dfTpm.sort_values(by='Confirmed', ascending=False)
dfTpm

,Code,Confirmed
9,Mainland China,6070
16,Thailand,14
6,Hong Kong,10
15,Taiwan,8
10,Malaysia,7
12,Singapore,7
7,Japan,7
8,Macau,7
17,US,5
0,Australia,5


In [93]:
fig = go.Figure(data=go.Choropleth(
    locations = dfTpm['Code'],
    z = dfTpm['Confirmed'],
    text = dfTpm['Code'],
    colorscale = 'Reds',
    autocolorscale=False,
    reversescale=False,
    colorbar_title = 'Confirmed cases',
    hovertemplate = "<b>%{text}</b><br><br>" +
                        "Confirmed: %{z}<br>" +
                        "<extra></extra>"# "<extra></extra>" means we don't display the info in the secondary box, such as trace id.

))
                
fig.update_layout(
    plot_bgcolor='#1f2630',
    paper_bgcolor='#1f2630',
    font=dict(color='#2cfec1'),
    geo=dict(showframe=True,
             showcoastlines=True,
             projection_type='equirectangular'),
    annotations = [dict(
        x=0.95,
        y=0,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://3g.dxy.cn/newh5/view/pneumonia?scene=2&clicktime=1579582238&enterid=1579582238&from=singlemessage&isappinstalled=0">Dingxiangyuan</a>',
        showarrow = False
    )]
)

fig.show()

In [63]:
import dash
import dash_core_components as dcc
import dash_html_components as html 

In [79]:
app = dash.Dash(__name__, assets_folder='./assets/')

In [134]:
app.layout = html.Div([
        html.Div(
            id='header',
            html.H2('Wuhan Coronavirus (2019-nCoV) Global Cases'),
            style={'width': '48%', 'display': 'inline-block'}
        ),
        html.Div(
            id='numberBlock',
            gtml.H4("Heatmap of age adjusted mortality rates from poisonings in year {0}".format(min(dfTpm['Confirmed'])),
            style={'width': '48%', 'display': 'inline-block'}
        ),
        html.Div(
           id='linePlot',
           [dcc.Graph(
               figure = {'data':[{'x': dfTpm['Code'], 'y': dfTpm['Confirmed'], 'type':'line', 'name' : '1st Chart'}],
                         'layout':{'plot_bgcolor':'#1f2630','paper_bgcolor':'#1f2630',
                                   'font' :{'color':'#2cfec1'},
                                   'title' : 'Simple Bar Chart'}})] 
        ),
        html.Div(dcc.Graph(id = 'map', figure =fig))
        
])

NameError: name 'well_status_options' is not defined

In [133]:
if __name__ == '__main__':
    app.run_server(port=8882)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8882/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jan/2020 19:16:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 19:16:22] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 19:16:22] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 19:16:22] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.3.1&m=1576595950 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 19:16:22] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 19:16:22] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.1&m=1576596177 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 19:16:22] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 1